In [1]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.driver.memory", "16g"). ,→appName('chapter_8').getOrCreate()
stocks = spark.read.csv(["data/stocksA/ABAX.csv","data/stocksA/AAME.csv","data/ ,→stocksA/AEPI.csv"], header='true', inferSchema='true')
from pyspark.sql.functions import col
df = df.withColumn("Open", col("Open").cast("double "))
df = df.withColumn("High", col("High").cast("double "))
df = df.withColumn("Low", col("Low").cast("double "))
df = df.withColumn("Volume", col("Volume").cast("double "))
df.printSchema()
import pyspark.sql.functions as f
params={}
for col in df.columns:
  if col!='Date':
    mean=df.agg(f.avg(col)).collect()[0][0]
    std=df.agg(f.stddev(col)).collect()[0][0]
    params[col]=[mean,std]

for key,val in params.items():
  print(key,val)
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import numpy as np
num_simulations = 1000

@udf(FloatType())
def open():
  return np.random.normal(params['Open'][0],params['Open'][0])

@udf(FloatType())
def High():
  return np.random.normal(params['High'][0],params['High'][0])

@udf(FloatType())
def Low():
  return np.random.normal(params['Low'][0],params['Low'][0])

@udf(FloatType())
def Close():
  return np.random.normal(params['Close'][0],params['Close'][0])

@udf(FloatType())
def Volume():
  return np.random.normal(params['Volume'][0],params['Volume'][0])
simulated_prices_df = df.select(
    open().alias("simulated_open"),
    Close().alias("simulated_close"),
    High().alias("simulated_high"),
    Low().alias("simulated_low"),
    Volume().alias("simulated_vol")
)
simulated_prices_df.show()